# CSV to JSON Converter

Convert the latest random walk and GP scaling experiment CSV files to JSON format for easier data processing and analysis.

In [4]:
import pandas as pd
import json
import os
import glob
import re
from datetime import datetime

# Set up paths
stats_dir = os.path.join(os.getcwd(), 'stats')
print(f"Looking for CSV files in: {stats_dir}")

if not os.path.exists(stats_dir):
    print("❌ Stats directory not found")
else:
    print(f"✅ Stats directory found")

Looking for CSV files in: /scratches/cartwright/mz473/Efficient-Gaussian-Process-on-Graphs/experiments_sparse/scaling_exp/stats
✅ Stats directory found


## Helper Functions

In [5]:
def extract_timestamp_from_filename(filename):
    """Extract timestamp from filename like 'file_stats_20241201_143052.csv'"""
    timestamp_pattern = r'_(\d{8}_\d{6})\.csv$'
    match = re.search(timestamp_pattern, filename)
    return match.group(1) if match else datetime.now().strftime("%Y%m%d_%H%M%S")

def find_latest_csv_files(stats_dir):
    """Find the most recent CSV files in the stats directory"""
    file_patterns = {
        'rw_sampling': 'rw_sampling_stats*.csv',
        'sparse_gp': 'sparse_gp_scaling_stats*.csv', 
        'dense_gp': 'dense_gp_scaling_stats*.csv'
    }
    
    latest_files = {}
    for file_type, pattern in file_patterns.items():
        files = glob.glob(os.path.join(stats_dir, pattern))
        if files:
            latest_file = max(files, key=os.path.getmtime)
            latest_files[file_type] = latest_file
            mod_time = datetime.fromtimestamp(os.path.getmtime(latest_file))
            print(f"Found {file_type}: {os.path.basename(latest_file)} (modified: {mod_time})")
        else:
            print(f"⚠️  No files found for {file_type}")
    
    return latest_files

def csv_to_json(csv_file, output_file):
    """Convert CSV to JSON with metadata"""
    df = pd.read_csv(csv_file)
    file_stat = os.stat(csv_file)
    
    json_data = {
        'metadata': {
            'source_file': os.path.basename(csv_file),
            'conversion_timestamp': datetime.now().isoformat(),
            'source_modified': datetime.fromtimestamp(file_stat.st_mtime).isoformat(),
            'total_records': len(df),
            'columns': list(df.columns),
            'dtypes': {col: str(df[col].dtype) for col in df.columns}
        },
        'data': df.to_dict(orient='records')
    }
    
    with open(output_file, 'w') as f:
        json.dump(json_data, f, indent=2, default=str)
    
    print(f"✅ Converted {os.path.basename(csv_file)} -> {os.path.basename(output_file)}")
    return json_data

def load_json_to_dataframes(json_file):
    """Load JSON file and return DataFrames with proper dtypes"""
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    dataframes = {}
    
    if 'datasets' in data:
        # Combined format
        for exp_type, exp_data in data['datasets'].items():
            dataframes[exp_type] = pd.DataFrame(exp_data['data'])
    else:
        # Single dataset format - infer type from filename
        df = pd.DataFrame(data['data'])
        exp_type = 'data'
        if 'metadata' in data and 'source_file' in data['metadata']:
            source = data['metadata']['source_file']
            if 'rw_sampling' in source:
                exp_type = 'rw_sampling'
            elif 'sparse_gp' in source:
                exp_type = 'sparse_gp'
            elif 'dense_gp' in source:
                exp_type = 'dense_gp'
        dataframes[exp_type] = df
    
    return dataframes

print("✅ Helper functions loaded")

✅ Helper functions loaded


## Convert CSV to JSON

In [6]:
# Find and convert CSV files
latest_files = find_latest_csv_files(stats_dir)

if not latest_files:
    print("❌ No CSV files found to convert")
else:
    converted_files = []
    
    # Convert individual files (preserving original timestamp)
    for file_type, csv_file in latest_files.items():
        timestamp = extract_timestamp_from_filename(csv_file)
        json_filename = f"{file_type}_stats_{timestamp}.json"
        json_filepath = os.path.join(stats_dir, json_filename)
        
        json_data = csv_to_json(csv_file, json_filepath)
        converted_files.append({
            'type': file_type,
            'csv_file': csv_file,
            'json_file': json_filepath,
            'timestamp': timestamp,
            'records': len(json_data['data'])
        })
    
    # Create combined master file (using latest timestamp)
    master_timestamp = max(cf['timestamp'] for cf in converted_files)
    master_data = {
        'conversion_info': {
            'timestamp': datetime.now().isoformat(),
            'total_files_converted': len(converted_files),
            'file_details': [{
                'type': cf['type'],
                'source': os.path.basename(cf['csv_file']),
                'output': os.path.basename(cf['json_file']),
                'record_count': cf['records']
            } for cf in converted_files]
        },
        'datasets': {}
    }
    
    # Load and combine all JSON files
    for cf in converted_files:
        with open(cf['json_file'], 'r') as f:
            master_data['datasets'][cf['type']] = json.load(f)
    
    # Save master file
    master_filename = f"scaling_experiment_combined_{master_timestamp}.json"
    master_filepath = os.path.join(stats_dir, master_filename)
    
    with open(master_filepath, 'w') as f:
        json.dump(master_data, f, indent=2, default=str)
    
    # Summary
    total_records = sum(cf['records'] for cf in converted_files)
    print(f"\n📊 CONVERSION SUMMARY")
    print(f"   Files processed: {len(converted_files)}")
    print(f"   Total records: {total_records}")
    print(f"   Master file: {master_filename}")
    print(f"   File size: {os.path.getsize(master_filepath) / 1024:.1f} KB")

Found rw_sampling: rw_sampling_stats_20250810_020458.csv (modified: 2025-08-10 02:04:58.802807)
Found sparse_gp: sparse_gp_scaling_stats_20250810_024906.csv (modified: 2025-08-10 02:49:07.095155)
Found dense_gp: dense_gp_scaling_stats_20250810_025004.csv (modified: 2025-08-10 02:50:04.380239)
✅ Converted rw_sampling_stats_20250810_020458.csv -> rw_sampling_stats_20250810_020458.json
✅ Converted rw_sampling_stats_20250810_020458.csv -> rw_sampling_stats_20250810_020458.json
✅ Converted sparse_gp_scaling_stats_20250810_024906.csv -> sparse_gp_stats_20250810_024906.json
✅ Converted dense_gp_scaling_stats_20250810_025004.csv -> dense_gp_stats_20250810_025004.json
✅ Converted sparse_gp_scaling_stats_20250810_024906.csv -> sparse_gp_stats_20250810_024906.json
✅ Converted dense_gp_scaling_stats_20250810_025004.csv -> dense_gp_stats_20250810_025004.json

📊 CONVERSION SUMMARY
   Files processed: 3
   Total records: 190
   Master file: scaling_experiment_combined_20250810_025004.json
   File siz

## Load JSON to DataFrames

In [9]:
# Load JSON to DataFrames for analysis
if 'master_filepath' in locals() and os.path.exists(master_filepath):
    print("📖 Loading master JSON to DataFrames...")
    
    # Load all experiments
    dfs = load_json_to_dataframes(master_filepath)
    
    # Extract individual DataFrames with clear variable names
    df_rw = dfs.get('rw_sampling', pd.DataFrame())
    df_sparse_gp = dfs.get('sparse_gp', pd.DataFrame()) 
    df_dense_gp = dfs.get('dense_gp', pd.DataFrame())
    
    # Display loading results
    print(f"✅ DataFrames loaded:")
    print(f"   df_rw        | {len(df_rw):4} records | {len(df_rw.columns):2} columns | Random Walk sampling results")
    print(f"   df_sparse_gp | {len(df_sparse_gp):4} records | {len(df_sparse_gp.columns):2} columns | Sparse GP experiment results")
    print(f"   df_dense_gp  | {len(df_dense_gp):4} records | {len(df_dense_gp.columns):2} columns | Dense GP experiment results")
    
    # Show sample data and column info
    print(f"\n📊 Column Information:")
    if len(df_rw) > 0:
        print(f"   df_rw columns: {list(df_rw.columns)}")
    if len(df_sparse_gp) > 0:
        print(f"   df_sparse_gp columns: {list(df_sparse_gp.columns)}")
    if len(df_dense_gp) > 0:
        print(f"   df_dense_gp columns: {list(df_dense_gp.columns)}")
    
    # Show sample records
    if len(df_rw) > 0:
        print(f"\n📊 Sample RW record:")
        sample_cols = list(df_rw.columns)[:5]
        for col in sample_cols:
            print(f"   {col:20} | {df_rw.iloc[0][col]}")
    
    # Quick analysis examples
    print(f"\n🔍 Quick Analysis Examples:")
    if len(df_rw) > 0:
        print(f"   Graph sizes in RW: {sorted(df_rw['n_nodes'].unique()) if 'n_nodes' in df_rw.columns else 'n_nodes column not found'}")
        if 'time_speedup' in df_rw.columns:
            print(f"   Average RW speedup: {df_rw['time_speedup'].mean():.2f}x")
    
    if len(df_sparse_gp) > 0 and len(df_dense_gp) > 0:
        if 'n_nodes' in df_sparse_gp.columns and 'n_nodes' in df_dense_gp.columns:
            sparse_sizes = set(df_sparse_gp['n_nodes'].unique())
            dense_sizes = set(df_dense_gp['n_nodes'].unique()) 
            common_sizes = sparse_sizes & dense_sizes
            print(f"   Common GP sizes: {sorted(common_sizes)}")
            
            if 'test_rmse' in df_sparse_gp.columns and 'test_rmse' in df_dense_gp.columns:
                sparse_rmse_avg = df_sparse_gp['test_rmse'].mean()
                dense_rmse_avg = df_dense_gp['test_rmse'].mean()
                print(f"   Average RMSE - Sparse: {sparse_rmse_avg:.4f}, Dense: {dense_rmse_avg:.4f}")
    
    print(f"\n🎯 DataFrames ready for analysis! Use df_rw, df_sparse_gp, df_dense_gp for pandas operations.")
    
else:
    print("⚠️ No master file available. Run the conversion first.")
    # Create empty DataFrames as fallback
    df_rw = pd.DataFrame()
    df_sparse_gp = pd.DataFrame() 
    df_dense_gp = pd.DataFrame()
    print("   Created empty DataFrames: df_rw, df_sparse_gp, df_dense_gp")

📖 Loading master JSON to DataFrames...
✅ DataFrames loaded:
   df_rw        |   80 records | 19 columns | Random Walk sampling results
   df_sparse_gp |   80 records | 10 columns | Sparse GP experiment results
   df_dense_gp  |   30 records | 10 columns | Dense GP experiment results

📊 Column Information:
   df_rw columns: ['n_nodes', 'n_edges', 'seed', 'sparse_rw_time', 'dense_rw_time', 'sparse_size_mb', 'dense_size_mb', 'sparse_dense_equiv_mb', 'compression_ratio', 'time_speedup', 'sparse_file_size_mb', 'dense_file_size_mb', 'sparse_total_nnz', 'sparse_avg_nnz_per_matrix', 'graph_sparsity', 'step_matrix_sparsity', 'run_dense', 'is_feasible', 'has_dense_data']
   df_sparse_gp columns: ['n_nodes', 'seed', 'n_train', 'n_test', 'train_time', 'inference_time', 'total_time', 'test_rmse', 'noise_variance', 'modulator_l2']
   df_dense_gp columns: ['n_nodes', 'seed', 'n_train', 'n_test', 'train_time', 'inference_time', 'total_time', 'test_rmse', 'noise_variance', 'modulator_l2']

📊 Sample RW 

In [22]:
df_rw[0:30]

,n_nodes,n_edges,seed,sparse_rw_time,dense_rw_time,sparse_size_mb,dense_size_mb,sparse_dense_equiv_mb,compression_ratio,time_speedup,sparse_file_size_mb,dense_file_size_mb,sparse_total_nnz,sparse_avg_nnz_per_matrix,graph_sparsity,step_matrix_sparsity,run_dense,is_feasible,has_dense_data
0,32,32,42,0.233220,0.159231,0.004395,0.023438,0.023438,5.333333,0.682751,0.004368,0.023706,288,96.0,0.062500,0.093750,True,True,True
1,32,32,43,0.152297,0.130707,0.004395,0.023438,0.023438,5.333333,0.858238,0.004368,0.023706,288,96.0,0.062500,0.093750,True,True,True
2,32,32,44,0.159181,0.105369,0.004395,0.023438,0.023438,5.333333,0.661944,0.004368,0.023706,288,96.0,0.062500,0.093750,True,True,True
3,32,32,45,0.138803,0.107336,0.004395,0.023438,0.023438,5.333333,0.773299,0.004368,0.023706,288,96.0,0.062500,0.093750,True,True,True
4,32,32,46,0.155783,0.102720,0.004395,0.023438,0.023438,5.333333,0.659379,0.004368,0.023706,288,96.0,0.062500,0.093750,True,True,True
5,64,64,42,0.219501,0.196751,0.008789,0.093750,0.093750,10.666667,0.896357,0.008043,0.094028,576,192.0,0.031250,0.046875,True,True,True
6,64,64,43,0.152514,0.196547,0.008789,0.093750,0.093750,10.666667,1.288711,0.008043,0.094028,576,192.0,0.031250,0.046875,True,True,True
7,64,64,44,0.169220,0.197837,0.008789,0.093750,0.093750,10.666667,1.169114,0.008043,0.094028,576,192.0,0.031250,0.046875,True,True,True
8,64,64,45,0.167746,0.200013,0.008789,0.093750,0.093750,10.666667,1.192358,0.008043,0.094028,576,192.0,0.031250,0.046875,True,True,True
9,64,64,46,0.168730,0.194206,0.008789,0.093750,0.093750,10.666667,1.150983,0.008043,0.094028,576,192.0,0.031250,0.046875,True,True,True


In [11]:
df_dense_gp

,n_nodes,seed,n_train,n_test,train_time,inference_time,total_time,test_rmse,noise_variance,modulator_l2
0,32,42,19,7,2.542858,0.018328,2.561186,0.279104,0.000001,0.627524
1,32,43,19,7,1.586494,0.014647,1.601141,0.319528,0.000001,0.692777
2,32,44,19,7,1.188662,0.014519,1.203181,0.342918,0.000001,0.637213
3,32,45,19,7,1.984586,0.014566,1.999151,0.260765,0.000001,0.658263
4,32,46,19,7,1.465889,0.021205,1.487093,0.293532,0.000001,0.612269
5,64,42,38,14,1.757632,0.017867,1.775498,0.183118,0.000001,0.584356
6,64,43,38,14,1.505821,0.025490,1.531312,0.194124,0.000001,0.552594
7,64,44,38,14,1.183140,0.014714,1.197854,0.153354,0.000001,0.551589
8,64,45,38,14,1.195086,0.014579,1.209665,0.172860,0.000001,0.561653
9,64,46,38,14,1.322639,0.014821,1.337460,0.156335,0.000001,0.581909


In [19]:
df_sparse_gp[-20:]

,n_nodes,seed,n_train,n_test,train_time,inference_time,total_time,test_rmse,noise_variance,modulator_l2
60,131072,42,78643,26215,35.098325,0.697238,35.795564,0.609307,0.036150,2.727336
61,131072,43,78643,26215,37.366946,0.816869,38.183815,0.609194,0.015161,2.704241
62,131072,44,78643,26215,23.490099,0.385458,23.875558,0.671547,0.474637,3.051420
63,131072,45,78643,26215,25.842498,0.574860,26.417358,0.599769,0.068337,2.460862
64,131072,46,78643,26215,35.870232,0.781828,36.652059,0.612035,0.025194,2.785411
65,262144,42,157286,52430,46.688528,0.853557,47.542085,0.603889,0.151859,2.481155
66,262144,43,157286,52430,45.618595,0.748653,46.367248,0.764697,0.813733,2.828024
67,262144,44,157286,52430,50.268165,1.122470,51.390635,0.602529,0.108417,2.884240
68,262144,45,157286,52430,80.998858,1.705197,82.704055,0.616033,0.016018,2.589691
69,262144,46,157286,52430,78.864747,1.648549,80.513296,0.615804,0.017587,2.770270
